In [1]:
import pandas as pd
import numpy as np

In [2]:
CLIENT_ID = 'QMG2AOEN44O0JRQDHT0XGLWSLZ54YEUEDPMWSJO31KQZHCPR' # your Foursquare ID
CLIENT_SECRET = 'VKOW3UW1SHZLS45ZGNHSB4E4MB1TMTE4YIYKHQFTLJUU1SGH' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QMG2AOEN44O0JRQDHT0XGLWSLZ54YEUEDPMWSJO31KQZHCPR
CLIENT_SECRET:VKOW3UW1SHZLS45ZGNHSB4E4MB1TMTE4YIYKHQFTLJUU1SGH


In [3]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [4]:
address = 'Chiang Mai, TH'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Chiang Mai, TH are 18.7905618, 98.9880909.


In [5]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QMG2AOEN44O0JRQDHT0XGLWSLZ54YEUEDPMWSJO31KQZHCPR&client_secret=VKOW3UW1SHZLS45ZGNHSB4E4MB1TMTE4YIYKHQFTLJUU1SGH&v=20180604&ll=18.7905618,98.9880909&radius=500&limit=100'

In [6]:
from pandas.io.json import json_normalize

In [7]:
import requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c543bc8f594df2114081fc5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ccb8b6fc9b846888ed0b6c3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_',
          'suffix': '.png'},
         'id': '52e81612bcbc57f1066b7a3e',
         'name': 'Buddhist Temple',
         'pluralName': 'Buddhist Temples',
         'primary': True,
         'shortName': 'Buddhist Temple'}],
       'id': '4ccb8b6fc9b846888ed0b6c3',
       'location': {'address': '3 Kings Square',
        'cc': 'TH',
        'city': 'เมืองเชียงใหม่',
        'country': 'ประเทศไทย',
        'crossStreet': 'Inthawarorot Rd.',
        'distance': 134,
        'formattedAddress': ['3 Kings Square (Inthawarorot Rd.)',
         'เมืองเชียงใหม่',
         'เชียงใ

In [ ]:
len(results['response']['groups'][0]['items'])

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,City Pillar Temple (วัดอินทขีลสะดือเมือง),Buddhist Temple,18.789773,98.987125
1,ก๋วยจั๊บน้ำข้น สามกษัตริย์,Noodle House,18.791136,98.986062
2,Kiat Ocha (เกียรติโอชา),Asian Restaurant,18.789709,98.986385
3,ห้องสมุดฟื้นบ้านย่านเวียงเชียงใหม่,Library,18.790276,98.986094
4,Yentafo Sa-ard (เย็นตาโฟสอาด),Noodle House,18.789775,98.986441


In [10]:
nearby_venues.shape

(78, 4)

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [14]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY
nearby_venues[nearby_venues['categories'] == 'Coffee Shop']

,name,categories,lat,lng
10,Ponganes Espresso (ปองกาเนส เอสเปรสโซ),Coffee Shop,18.790051,98.990077
15,Pacamara Boutique Coffee Roasters (ร้านกาแฟ พา...,Coffee Shop,18.788202,98.986746
19,Graph Café (กราฟ คาเฟ่),Coffee Shop,18.791736,98.991520
27,Bartcoffee,Coffee Shop,18.792250,98.991035
55,Cafe Arte (คาเฟ่ อาร์เต้),Coffee Shop,18.792727,98.992210
65,RockPresso Cafe & House (ร็อคเพรสโซ คาเฟ่ แอนด...,Coffee Shop,18.793242,98.988146


In [ ]:
map_chiangmai = folium.Map(location=[latitude, longitude], zoom_start=16)

# add markers to map
for name, cate, lat, lng in zip(nearby_venues['name'], nearby_venues['categories'], nearby_venues['lat'], nearby_venues['lng']):
    label = '{} - {}'.format(name, cate)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chiangmai)  
    
map_chiangmai

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
chiangmai_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

City Pillar Temple (วัดอินทขีลสะดือเมือง)
ก๋วยจั๊บน้ำข้น สามกษัตริย์
Kiat Ocha (เกียรติโอชา)
ห้องสมุดฟื้นบ้านย่านเวียงเชียงใหม่
Yentafo Sa-ard (เย็นตาโฟสอาด)
Chiangmai Walking Street (ถนนคนเดินเชียงใหม่)
Mana Sticky Rice (ข้าวเหนียวมานะ)
Cozytel
U Chiang Mai Hotel
Tamarind Village Hotel Chiang Mai
Ponganes Espresso (ปองกาเนส เอสเปรสโซ)
Archers Restaurant
Fah Lanna Spa
Krua Dabb Lob (ครัวดาบลพ)
Khunka Massage
Pacamara Boutique Coffee Roasters (ร้านกาแฟ พาคามาร่า)
ข้าวหมูกรอบ 3 กษัตริย์
Chiang Mai Women's Prison Massage Centre
Vigie Sist Cafe
Graph Café (กราฟ คาเฟ่)
Yok Fa Pochana (หยกฟ้าโภชนา)
Tamarind Restaurant
Chiang Mai City Arts & Cultural Centre (หอศิลปวัฒนธรรมเมืองเชียงใหม่)
ช้างม่อยกาแฟ
Fahtara Coffee
Lamphu House
Farm Story House (ฟาร์ม สตอรี่ เฮาส์)
Bartcoffee
Wanlamun (หวานละมุน)
Chiang Mai Historical Centre (หอประวัติศาสตร์เมืองเชียงใหม่)
By Hand Pizza Café
Wat Chiang Man (วัดเชียงมั่น)
Three Kings Monument (พระบรมราชานุสาวรีย์สามกษัตริย์)
Khun Kae Juice Bar
Roots Rock Regga

In [17]:
chiangmai_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Chiangmai Walking Street (ถนนคนเดินเชียงใหม่),18.788177,98.988072,Night Market
1,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,Noodle House
2,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Krua Dabb Lob (ครัวดาบลพ),18.788336,98.985597,Thai Restaurant
3,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Buddhist Temple
4,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,ห้องสมุดฟื้นบ้านย่านเวียงเชียงใหม่,18.790276,98.986094,Library
5,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Kiat Ocha (เกียรติโอชา),18.789709,98.986385,Asian Restaurant
6,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,U Chiang Mai Hotel,18.788485,98.987138,Hotel
7,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Yentafo Sa-ard (เย็นตาโฟสอาด),18.789775,98.986441,Noodle House
8,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Khunka Massage,18.788234,98.986681,Spa
9,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Pacamara Boutique Coffee Roasters (ร้านกาแฟ พา...,18.788202,98.986746,Coffee Shop


In [18]:
chiangmai_venues[chiangmai_venues['Venue Category'] == 'Coffee Shop']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Pacamara Boutique Coffee Roasters (ร้านกาแฟ พา...,18.788202,98.986746,Coffee Shop
26,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Akha Ama Coffee La Fattoria,18.788393,98.983314,Coffee Shop
27,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Ponganes Espresso (ปองกาเนส เอสเปรสโซ),18.790051,98.990077,Coffee Shop
42,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,Bartcoffee,18.792250,98.991035,Coffee Shop
65,City Pillar Temple (วัดอินทขีลสะดือเมือง),18.789773,98.987125,RockPresso Cafe & House (ร็อคเพรสโซ คาเฟ่ แอนด...,18.793242,98.988146,Coffee Shop
89,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,Pacamara Boutique Coffee Roasters (ร้านกาแฟ พา...,18.788202,98.986746,Coffee Shop
93,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,Akha Ama Coffee La Fattoria,18.788393,98.983314,Coffee Shop
115,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,Ponganes Espresso (ปองกาเนส เอสเปรสโซ),18.790051,98.990077,Coffee Shop
130,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,Mingmitr Coffee (มิ่งมิตร),18.795346,98.987378,Coffee Shop
132,ก๋วยจั๊บน้ำข้น สามกษัตริย์,18.791136,98.986062,RockPresso Cafe & House (ร็อคเพรสโซ คาเฟ่ แอนด...,18.793242,98.988146,Coffee Shop
